In [4]:
import pandas as pd
import sqlite3

path = '../../Welcome-Centre-DataCorps-Data/ClientDatabaseStructure.mdb.sqlite'
con = sqlite3.connect(path)

sql = """select * from Referral limit 10;"""
df = pd.read_sql(sql,con)

In [53]:
referrals = pd.read_sql("""select * from Referral
left join ReferralStatus on ReferralStatus.ReferralStatusId = Referral.StatusId
left join (
  select * from Client
left join Ethnicity on Client.ClientEthnicityID = Ethnicity.EthnicityID
left join Country on Client.ClientCountryID = Country.CountryId
left join ResidencyStatus on Client.ClientResidencyId = ResidencyStatus.ResidencyId
left join ClientAddressType on Client.ClientAddressTypeID = ClientAddressType.ClientAddressTypeID
left join Locality on Client.AddressLocalityId = Locality.LocalityId
  ) as client on client.ClientId = Referral.ClientId

left join Ethnicity on Referral.EthnicityId = Ethnicity.EthnicityID
left join Locality on Referral.AddressLocalityId = Locality.LocalityId
left join ClientAddressType on Referral.AddressTypeId = ClientAddressType.ClientAddressTypeID
left join ReferralAgency on Referral.ReferralAgencyId = ReferralAgency.ReferralAgencyId;""", con)
referrals = referrals.set_index('ReferralInstanceId')

In [20]:
table_sql = """select * from ReferralBenefit
  inner join BenefitType on BenefitType.BenefitTypeId = ReferralBenefit.BenefitTypeId;
select * from ReferralDietaryRequirements
  inner join DietaryRequirements on ReferralDietaryRequirements.DietaryRequirementsID = DietaryRequirements.DietaryRequirementsID;
select * from ReferralDocument
  inner join DocumentEvidence on ReferralDocument.ReferralDocumentId = DocumentEvidence.DocumentEvidenceId;
select * from ReferralDomesticCircumstances
  inner join DomesticCircumstances on DomesticCircumstances.DomesticCircumstancesID = ReferralDomesticCircumstances.DomesticCircumstancesID;
select * from ReferralIssue
  inner join ClientIssueCodes on ClientIssueCodes.ClientIssueId = ReferralIssue.ClientIssueID;
select * from ReferralReason
  inner join ReferralReasonCodes on ReferralReasonCodes.ReferralReasonId = ReferralReason.ReferralReasonID;"""
table_names = [t.split(' ')[3].strip() for t in table_sql.split(';')[:-1]]
tables = {table_names[i]: pd.read_sql(t.strip(), con) for i, t in enumerate(table_sql.split(';')[:-1])}

In [44]:
column_mapping = {
    'ReferralIssue': 'ClientIssueDescription',
    'ReferralBenefit': 'BenefitTypeName',
    'ReferralReason': 'ReferralReasonDescription',
    'ReferralDietaryRequirements': 'DietaryRequirementsDescription',
    'ReferralDomesticCircumstances': 'DomesticCircumstancesDescription',
    'ReferralDocument': 'DocumentEvidenceDescription'
}

In [46]:
flat_tables = {t: tables[t].groupby([tables[t].iloc[:,0], column_mapping[t]]).size().unstack() for t in tables}

In [54]:
for t in flat_tables:
    referrals = referrals.merge(flat_tables[t], left_index=True, right_index=True, how='left')

In [63]:
referrals.shape

(17980, 204)

In [60]:
(referrals.count() / len(referrals)).sort_values()*100

DietaryNotes                             0.000000
OrderNo                                  0.000000
ReferralContactEmail                     0.000000
PreviousAddressList                      0.000000
AddressLine4                             0.000000
AddressLine2                             0.000000
AddressLine1                             0.000000
ClientNickName                           0.000000
OrderNo:3                                0.000000
ReferralAgencyNotes                      0.005562
AddressLine2                             0.011123
fleeing domestic violence                0.011123
Healthy Eating Vouchers                  0.011123
AddressLine4                             0.016685
AddressPostCode                          0.038932
AddressLocalityId                        0.044494
meltham food service delivery            0.050056
Benefits - New/Changed Benefits Cap      0.061179
Coeliac                                  0.061179
Vegan                                    0.061179
